In [1]:
from graph_miner import NetworkRepositoryGraphRepository

In [2]:
repository = NetworkRepositoryGraphRepository()

In [3]:
repository.retrieve_all()

ValueError: In the edge file was found the node 999949 which is not present in the given node file.

In [1]:
import pandas as pd
import numpy as np
from downloaders import BaseDownloader
from ensmallen_graph import EnsmallenGraph
from downloaders.extractors.zip_extraction import ZipExtractor
from tqdm.auto import tqdm
import os
import shutil
import compress_json

In [2]:
url = "http://nrvis.com/download/data/{}/{}.zip"

In [3]:
path = "bindings/python/ensmallen_graph/network_repository/{}.json"

In [4]:
graphs = pd.read_html("http://networkrepository.com/networks.php")[0]
graphs["url"] = [
    url.format(row["Type"], row["Graph Name"])
    for _, row in graphs.iterrows()
]
graphs.drop(
    columns=["Download", "Size"],
    inplace=True
)

In [6]:
downloader = BaseDownloader(
    target_directory="graphs",
    process_number=1,
    verbose=2,
    crash_early=True
)

In [ ]:
EnsmallenGraph.from_unsorted_csv()

In [7]:
already_skipped = set()

In [43]:
metadata = compress_json.local_load("metadata.json")

In [44]:
to_add_manually = {
    'imdb',
    'ip-trace',
    "web-ClueWeb09",
    "web-ClueWeb09-50m"
}

In [69]:
to_skip = {
    "ca-cit-HepPh",
    "ca-cit-HepTh",
    "cit-HepPh",
    "cit-HepTh",
    'SMS-A',
    'aves-sparrow-social',
    'aves-wildbird-network',
    'comm-linux-kernel-reply',
    'contacts-prox-high-school-2013-attr',
    'edit-enwikibooks',
    'fb-wosn-friends',
    'ia-chess',
    'ia-digg-reply',
    'ia-enron-email-all',
    'ia-enron-email-dynamic',
    'ia-enron-employees',
    'ia-escorts-dynamic',
    'ia-facebook-wall-wosn-dir',
    'ia-frwikinews-user-edits',
    'insecta-ant-colony1',
    'insecta-ant-colony2',
    'soc-epinions-trust-dir',
    'soc-flickr-growth',
    "soc-twitter",
    'rec-movielens-user-tag-10m'
}

In [72]:
for _, row in tqdm(graphs.iterrows(), total=len(graphs)):
    url = row.url
    graph_name = row["Graph Name"]
    if graph_name in metadata or graph_name in to_skip or graph_name in already_skipped or graph_name in to_add_manually:
        continue
    report = downloader.download(url)
    if "extraction_success" not in report.columns:
        already_skipped.add(graph_name)
        continue
    extraction = report.extraction_destination[0]
    graph_name = extraction.split(os.sep)[-1]
    edge_path_pattern = "{}/{}".format(
        report.extraction_destination[0],
        graph_name
    )
    edge_path = None
    for ext in ("edges", "mtx", "txt"):
        edge_path = "{}.{}".format(edge_path_pattern, ext)
        if os.path.exists(edge_path):
            break
    if not os.path.exists(edge_path):
        already_skipped.add(graph_name)
        continue
    if edge_path is None:
        raise ValueError("File not found in list {}", os.listdir(extraction))
    comment = None
    if ext == "mtx":
        edge_rows_to_skip = 1
    else:
        edge_rows_to_skip = 0
        
    with open(edge_path, "r") as f:
        first_line = f.readline()
        
    for comment_symbol in ("%", "#"):
        if first_line.startswith(comment_symbol):
            comment = comment_symbol

    comment_lines = 0
    with open(edge_path, "r") as f:
        for _ in range(1000):
            middle_line = f.readline()
            if comment:
                if middle_line.startswith(comment):
                    comment_lines+=1
                
    for separator in ("\t", " ", ",", ";"):
        if separator in middle_line:
            edge_separator = separator
            
    data = pd.read_csv(
        edge_path,
        sep=edge_separator,
        skiprows=edge_rows_to_skip + comment_lines,
        header=None,
        nrows=20000
    )
    
    if len(data.columns) == 1:
        already_skipped.add(graph_name)
        continue
    
    if len(data) < 10:
        already_skipped.add(graph_name)
        continue
    
    has_index = False
    if len(data.columns) > 2 and (data[0].values == np.arange(len(data))).all():
        # This file has a damn index!
        sources_column_number = 1
        destinations_column_number = 2
        weights_column_number = 3
        has_index = True
    else:
        sources_column_number = 0
        destinations_column_number = 1
        weights_column_number = 2
        
    if len(data.columns) > weights_column_number + 1:
        if graph_name.startswith("ia-") or graph_name.startswith('insecta-ant') or graph_name.startswith("mammalia-"):
            already_skipped.add(graph_name)
            continue
        if graph_name.startswith("rec-"):
            to_add_manually.add(graph_name)
            continue
        already_skipped.add(graph_name)
        continue
        #raise ValueError("This graph has a lot of columns, check what is happening!")
        
    has_weights = False
    has_nan_weights = False
    if len(data.columns) > weights_column_number:
        has_weights = True
        if data[weights_column_number].isna().all():
            has_weights = False
        elif data[weights_column_number].isna().any():
            has_nan_weights=True
    
    if has_weights and (data[weights_column_number] <= 0).any():
        # We do not support graphs with negative weights.
        already_skipped.add(graph_name)
        continue
    
    graph = EnsmallenGraph.from_unsorted_csv(
        edge_path=edge_path,
        directed=False,
        edge_separator=edge_separator,
        sources_column_number=sources_column_number,
        destinations_column_number=destinations_column_number,
        edge_rows_to_skip=edge_rows_to_skip,
        edge_max_rows_number = 20000,
        **(dict(weights_column_number=weights_column_number) if has_weights else {}),
        **(dict(default_weight=1) if has_nan_weights else {}),
        **(dict(edge_file_comment_symbol=comment) if comment is not None else {})
    )
    metadata[graph_name] = {
        "urls": [url],
        "arguments": {
            "edge_path": "{graph_name}/{graph_name}.{ext}".format(
                graph_name=graph_name,
                ext=ext
            ),
            "edge_separator": edge_separator,
            "sources_column_number": sources_column_number,
            "destinations_column_number": destinations_column_number,
            **({
                "weights_column_number": 2,
            } if len(data.columns) > 2 else {}),
            **({
                "edge_rows_to_skip": edge_rows_to_skip,
            } if edge_rows_to_skip else {}),
            **({
                "edge_file_comment_symbol": comment
            } if comment is not None else {}),
            "edge_header": False,
            "has_nan_weights": has_nan_weights
        }
    }
    #os.remove(report.destination[0])
    #shutil.rmtree(report.extraction_destination[0])

In [75]:
metadata

{'bio-CE-CX': {'urls': ['http://nrvis.com/download/data/bio/bio-CE-CX.zip'],
  'arguments': {'edge_path': 'bio-CE-CX/bio-CE-CX.edges',
   'edge_separator': ' ',
   'sources_column_number': 0,
   'destinations_column_number': 1,
   'weights_column_number': 2,
   'edge_header': False,
   'numeric_edge_node_ids': True,
   'has_nan_weights': False}},
 'bio-CE-GT': {'urls': ['http://nrvis.com/download/data/bio/bio-CE-GT.zip'],
  'arguments': {'edge_path': 'bio-CE-GT/bio-CE-GT.edges',
   'edge_separator': ' ',
   'sources_column_number': 0,
   'destinations_column_number': 1,
   'weights_column_number': 2,
   'edge_header': False,
   'numeric_edge_node_ids': True,
   'has_nan_weights': False}},
 'bio-CE-HT': {'urls': ['http://nrvis.com/download/data/bio/bio-CE-HT.zip'],
  'arguments': {'edge_path': 'bio-CE-HT/bio-CE-HT.edges',
   'edge_separator': ' ',
   'sources_column_number': 0,
   'destinations_column_number': 1,
   'weights_column_number': 2,
   'edge_header': False,
   'numeric_edge_n

In [53]:
middle_line, first_line, graph_name

('', '%%MatrixMarket matrix coordinate pattern general\n', 'Trec3')

In [74]:
compress_json.local_dump(metadata, "metadata2.json")

In [ ]:
comment, edge_rows_to_skip, has_nan_weights

In [ ]:
middle_line

In [ ]:
os.listdir("graphs/ca-MathSciNet/")

In [ ]:
pd.read_csv("graphs/ca-MathSciNet/ca-MathSciNet.mtx", sep=" ", comment="%", skiprows=2, header=None)